In [2]:
from ECKG.src.baseline import BaselineController, MentionPairFeatures
from contextual_model_bert import ContextualControllerBERT
from data import read_classla, read_corpus
from eventify import *
from helper_functions import *
from books.get_data import get_data
import urllib.request

corpus = get_data("../../books/corpus.tsv", get_text=True)
sl_text  = "Ljubljana - Evropska komisija je Sloveniji za letos napovedala 1,9 - odstotno rast , za naslednje leto pa 2,5 - odstotno , kar je nad povprečjem v območju evra in EU . Sicer pa bo Slovenija letos po napovedih iz Bruslja beležila 5,8 - odstotni javnofinančni primanjkljaj , naslednje leto pa petodstotnega , kar je slabše od povprečja v območju evra in EU . Javni dolg Slovenije se bo po napovedih komisije vztrajno večal , a bo še vedno bistveno pod povprečjem območja evra in EU ter bistveno pod referenčno mejo 60 odstotkov bruto domačega proizvoda ( BDP ) , ki jo določa pakt za stabilnost in rast . Za letos komisija Sloveniji napoveduje 42,8 - odstotni dolg , za prihodnje leto pa 46 - odstotnega . Poleg tega Evropska komisija Sloveniji za letos napoveduje 2,6 - odstotno inflacijo , za prihodnje leto pa 2,1 - odstotno . Inflacija v Sloveniji bo tako letos po napovedih komisije enaka kot v povprečju v območju evra in pod povprečjem EU , prihodnje leto pa nekoliko nad povprečjem v območju evra in celotne EU . Po napovedih iz Bruslja bo Slovenija letos beležila večjo brezposelnost kot lani . Medtem ko je bila ta lani po ocenah komisije 7,3 - odstotna , naj bi bila letos 8,2 - odstotna , prihodnje leto pa osemodstotna . To je še vedno manj kot v povprečju v območju evra in celotne EU , kjer naj bi bila letos 10 - odstotna , prihodnje leto pa 9,7 - odstotna oziroma 9,5 - odstotna in 9,1 - odstotna . V lanski novembrski gospodarski napovedi je komisija Sloveniji za letos prav tako napovedala 1,9 - odstotno rast , za leto 2012 pa 2,6 - odstotno , a njene napovedi za javnofinančni primanjkljaj Slovenije so bile novembra boljše kot danes . Novembra je namreč napovedala , da bo ta v Sloveniji letos 5,3 - odstoten , naslednje leto pa 4,7 - odstoten . Slovenija je sicer - kot velika večina držav EU - zaradi prekomernega javnofinančnega primanjkljaja v postopku Evropske komisije . Evropska komisija je kot rok za znižanje javnofinančnega primanjkljaja Slovenije pod tri odstotke BDP , kar kot zgornjo mejo določa pakt za stabilnost in rast , določila leto 2013 . Slovenija je po navedbah finančnega ministrstva v programu stabilnosti , ki ga je posredovala Evropski komisiji , zapisala , da za leto 2012 načrtuje znižanje primanjkljaja na 3,9 odstotka BDP . Na zvišanje letošnjega primanjkljaja bo sicer po navedbah komisije vplivala dokapitalizacija največje slovenske banke , Nove Ljubljanske banke . Glede primanjkljaja v letu 2012 je komisija zapisala , da je izračunan ob predpostavki , da ne bo dodatnih ukrepov . Glede nadaljnjih ukrepov za zajezitev izdatkov v letu 2012 sicer komisija ugotavlja , da ti v glavnem še niso podrobno opredeljeni in dogovorjeni . Glede gospodarske rasti pa Evropska komisija v primeru Slovenije ugotavlja razmeroma medlo okrevanje , ki temelji na izvozu . Ta pa je po njenih navedbah ohromljen zaradi izgub konkurenčnosti , ki so se nabrale pred in med krizo , ter zaradi usmerjenosti izvoza na gospodarsko šibek Zahodni Balkan . Drugi dejavnik , ki zavira gospodarsko rast Slovenije , so po navedbah Evropske komisije razmere v gradbeništvu . Poleg tega se je po navedbah komisije v prizadevanjih bank za izboljšanje svojih bilanc skrčil tudi tok posojil v realno gospodarstvo"


# Download text and decode
# Note: Set proxy if behind a proxy (https://docs.python.org/2/library/urllib.html)
url = "http://www.gutenberg.org/files/1342/1342-0.txt"
en_text = urllib.request.urlopen(url).read().decode()
en_text = ".".join(en_text.split(".")[20:50])



## Slovene Coreference resolution

In [18]:
model_name = "baseline_model_senticoref"
learning_rate = 0.05
dataset_name = "senticoref"
# BASELINE
model = BaselineController(MentionPairFeatures.num_features(),
                           model_name=model_name,
                           learning_rate=learning_rate,
                           dataset_name=dataset_name
                           )
model = BaselineController.from_pretrained("baseline_model/baseline_model_senticoref")

# CONTEXTUAL BERT
model = ContextualControllerBERT.from_pretrained("contextual_model_bert/fold7_0")

# read corpus
documents = read_corpus(dataset_name)

# test on first document
eval_doc = model._train_doc(documents[0], eval_mode=True)
print("ORIGINAL CLUSTERS")
for cluster in documents[0].clusters:
    print([" ".join([y.raw_text for y in documents[0].mentions[x].tokens]) for x in cluster])
print("PREDICTED CLUSTERS")
for key, values in eval_doc[0].items():
    if key is None:
        key = "None"
    try:
        print(" ".join([y.raw_text for y in documents[0].mentions[key].tokens]), ":")
        print([" ".join([y.raw_text for y in documents[0].mentions[x].tokens]) for x in values])
    except KeyError:
        continue
# Evaluation for conll format produced by classla
#     classla.download('sl')
#     pipeline = classla.Pipeline("sl", processors='tokenize,ner, lemma, pos, depparse', use_gpu=True)
#
#     # Analyze using classla/stanza to get Document format
#     data = pipeline(sl_text)
#     # Convert to conll format
#     data = data.to_conll()
#     # Convert to format used by slovene-coreference-resolution library (https://github.com/matejklemen/slovene-coreference-resolution)
#     # TODO: need to fix some mistakes when converting formats
#     data = read_classla(data)
    # res, _ = model._train_doc(data3, eval_mode=True)

INFO:root:Initialized baseline model with name baseline_model_senticoref
INFO:root:Initialized baseline model with name baseline_model_senticoref
Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ORIGINAL CLUSTERS
['Ljubljana']
['Evropska komisija', 'komisije', 'komisija', 'Evropska komisija', 'komisije', 'komisije', 'komisija', 'njene', 'napovedala', 'Evropske komisije', 'Evropska komisija', 'Evropski komisiji', 'komisija', 'komisija', 'Evropska komisija', 'njenih', 'Evropske komisije', 'komisije']
['Sloveniji', 'Slovenija', 'Slovenije', 'Sloveniji', 'Sloveniji', 'Sloveniji', 'Slovenija', 'Sloveniji', 'Slovenije', 'Sloveniji', 'Slovenija', 'Slovenije', 'Slovenija', 'posredovala', 'načrtuje', 'Slovenije', 'Slovenije']
['EU', 'EU', 'EU', 'EU', 'EU', 'EU', 'EU']
['Bruslja', 'Bruslja']
['največje slovenske banke', 'Nove Ljubljanske banke']
['Zahodni Balkan']
PREDICTED CLUSTERS
Sloveniji :
['Slovenija', 'Slovenije', 'Slovenija']
EU :
['EU', 'EU']
Evropska komisija :
['komisije', 'komisija', 'Evropska komisija']
Slovenija :
['Sloveniji', 'Sloveniji']
Sloveniji :
['Sloveniji', 'Slovenija', 'Slovenije']
komisije :
['komisije', 'komisija', 'njenih']
EU :
['EU']
EU :
['EU', 'Evropske ko

## NER + SVN triplet extraction

### Slovene

In [3]:
# Initialize Slovene classla pipeline
classla.download('sl')
sl_pipeline = classla.Pipeline("sl", processors='tokenize,ner, lemma, pos, depparse', use_gpu=True)
e = Eventify(language="sl")

INFO:classla:Downloading these customized packages for language: sl (Slovenian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

INFO:classla:File exists: /home/erikkastelec/classla_resources/sl/pos/standard.pt.
INFO:classla:File exists: /home/erikkastelec/classla_resources/sl/lemma/standard.pt.
INFO:classla:File exists: /home/erikkastelec/classla_resources/sl/depparse/standard.pt.
INFO:classla:File exists: /home/erikkastelec/classla_resources/sl/ner/standard.pt.
INFO:classla:File exists: /home/erikkastelec/classla_resources/sl/pretrain/standard.pt.
INFO:classla:Finished downloading models and saved to /home/erikkastelec/classla_resources.
INFO:classla:Loading these models for language: sl (Slovenian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| deppar

In [4]:

sl_text  = "Ljubljana - Evropska komisija je Sloveniji za letos napovedala 1,9 - odstotno rast , za naslednje leto pa 2,5 - odstotno , kar je nad povprečjem v območju evra in EU . Sicer pa bo Slovenija letos po napovedih iz Bruslja beležila 5,8 - odstotni javnofinančni primanjkljaj , naslednje leto pa petodstotnega , kar je slabše od povprečja v območju evra in EU . Javni dolg Slovenije se bo po napovedih komisije vztrajno večal , a bo še vedno bistveno pod povprečjem območja evra in EU ter bistveno pod referenčno mejo 60 odstotkov bruto domačega proizvoda ( BDP ) , ki jo določa pakt za stabilnost in rast . Za letos komisija Sloveniji napoveduje 42,8 - odstotni dolg , za prihodnje leto pa 46 - odstotnega . Poleg tega Evropska komisija Sloveniji za letos napoveduje 2,6 - odstotno inflacijo , za prihodnje leto pa 2,1 - odstotno . Inflacija v Sloveniji bo tako letos po napovedih komisije enaka kot v povprečju v območju evra in pod povprečjem EU , prihodnje leto pa nekoliko nad povprečjem v območju evra in celotne EU . Po napovedih iz Bruslja bo Slovenija letos beležila večjo brezposelnost kot lani . Medtem ko je bila ta lani po ocenah komisije 7,3 - odstotna , naj bi bila letos 8,2 - odstotna , prihodnje leto pa osemodstotna . To je še vedno manj kot v povprečju v območju evra in celotne EU , kjer naj bi bila letos 10 - odstotna , prihodnje leto pa 9,7 - odstotna oziroma 9,5 - odstotna in 9,1 - odstotna . V lanski novembrski gospodarski napovedi je komisija Sloveniji za letos prav tako napovedala 1,9 - odstotno rast , za leto 2012 pa 2,6 - odstotno , a njene napovedi za javnofinančni primanjkljaj Slovenije so bile novembra boljše kot danes . Novembra je namreč napovedala , da bo ta v Sloveniji letos 5,3 - odstoten , naslednje leto pa 4,7 - odstoten . Slovenija je sicer - kot velika večina držav EU - zaradi prekomernega javnofinančnega primanjkljaja v postopku Evropske komisije . Evropska komisija je kot rok za znižanje javnofinančnega primanjkljaja Slovenije pod tri odstotke BDP , kar kot zgornjo mejo določa pakt za stabilnost in rast , določila leto 2013 . Slovenija je po navedbah finančnega ministrstva v programu stabilnosti , ki ga je posredovala Evropski komisiji , zapisala , da za leto 2012 načrtuje znižanje primanjkljaja na 3,9 odstotka BDP . Na zvišanje letošnjega primanjkljaja bo sicer po navedbah komisije vplivala dokapitalizacija največje slovenske banke , Nove Ljubljanske banke . Glede primanjkljaja v letu 2012 je komisija zapisala , da je izračunan ob predpostavki , da ne bo dodatnih ukrepov . Glede nadaljnjih ukrepov za zajezitev izdatkov v letu 2012 sicer komisija ugotavlja , da ti v glavnem še niso podrobno opredeljeni in dogovorjeni . Glede gospodarske rasti pa Evropska komisija v primeru Slovenije ugotavlja razmeroma medlo okrevanje , ki temelji na izvozu . Ta pa je po njenih navedbah ohromljen zaradi izgub konkurenčnosti , ki so se nabrale pred in med krizo , ter zaradi usmerjenosti izvoza na gospodarsko šibek Zahodni Balkan . Drugi dejavnik , ki zavira gospodarsko rast Slovenije , so po navedbah Evropske komisije razmere v gradbeništvu . Poleg tega se je po navedbah komisije v prizadevanjih bank za izboljšanje svojih bilanc skrčil tudi tok posojil v realno gospodarstvo"

sl_text = "Nekoč je živela majhna deklica. Bila je tako zelo ljubka, da jo je vsak že na prvi pogled vzljubil." \
          " Izmedvseh ljudi na tem svetu pa jo je imela najraje njena babica. Ni je bilo stvari, ki je ne bi naredila zanjo.Nekega dne ji je podarila žametno rdečo kapico." \
          " Ta je bila deklici tako zelo všeč, da jo je nosila, kamorkoli je šla." \
          " Zato so jo vsi klicali Rdeča kapica.Nekega dne ji je mama rekla: \"Pridi, Rdeča kapica. Tukaj je kos potice in steklenica vina, nesi ju svoji babici." \
          " Zelo je bolna in slabotna. Okrepčilo ji bo dobro delo." \
          " Pa le hitro se odpravi, preden postane prevroče." \
          " Hodi previdno in tiho. Predvsem pa ne zahajaj s poti, da ne boš padla in razbila steklenice, ker potem babica ne bo dobila ničesar." \
          " Ko vstopiš v sobo, ne pozabi reči \"Dobro jutro,\" in ne preglej vsakega kotička preden to storiš." \
          "\"Pazljiva bom,\" ji je obljubila Rdeča kapica. Babica je živela globoko v gozdu, za dobro uro hoda stran." \
          " Ko je Rdeča kapica vstopila v gozd, ji je naproti stopil volk." \
          " Ker Rdeča kapica še nikdar ni srečala volka in ni vedela, kako zlobna in prebrisana je ta zver, se ga sploh ni bala." \
          "\"Pozdravljena, Rdeča kapica,\" je rekel volk.\
\"Hvala enako, prijazni volk.\"\
\"Kam pa kam tako zarana, Rdeča kapica?\"\
\"K babici.\"\
\"Kaj pa nosiš v košarici?\"\
\"Potico in vino, včeraj je mama pekla. Uboga bolna babica bo\
dobila nekaj dobrega za pod zob. Da se okrepča.\"\
\"Kje pa živi tvoja babica?\"\
\"Dobro uro hoda tja globoko v gozd. Njena hiša stoji pod tremi hrasti, in leske rastejo vse naokrog. Prav\
gotovo si jo že videl,\" je odgovorila Rdeča kapica.\
Volk si je mislil: \"Oh, kako okusna bo tale mala deklica, ko jo bom snedel. Precej boljša kot stara ženička.\
Ampak, če bom dovolj prebrisan, bom lahko pohrustal obe.\" Tako je volk del poti spremljal Rdečo kapico, preden se je spet oglasil: \"Poglej, Rdeča kapica, kako lepe rože rastejo povsod tu naokrog. In\
samo poslušaj, kako lepo pojejo vse te ptičice. Ti pa tako resno hodiš, kot bi bila na poti v šolo. Vse\
naokrog pa je tako čarobno.\" Medtem pa jo je volk po najkrajši možni poti ucvrl naravnost do babičine\
hiše in potrkal na vrata.\
\"Kdo je?\"\
\"Rdeča kapica. Prinesla sem ti potice in vina. Odpri mi vrata.\"\
\"Kar odpri si,\" je zaklicala babica. \"Preslabotna sem in ne morem vstati.\"\
Volk je pritisnil na kljuko in vrata so se na stežaj odprla. Brez besed je vstopil in planil naravnost na\
babičino posteljo. V enem samem grižljaju jo je požrl. Potem si je nadel babičina oblačila in poveznil\
nočno kapo na glavo. Zagrnil je zavese in se udobno ulegel v babičino posteljo.\
Rdeča kapica je ves ta čas tekala po gozdu in nabirala cvetlice. Ko jih je nabrala toliko, da jih je že komaj\
držala, se je spomnila na babico in se končno odpravila do nje.\
Bila je presenečena, ko je našla vrata babičine hiše na stežaj odprta. Ko je vstopila, jo je prevzel čuden občutek. Potiho si je rekla: \"Oh. Danes se pa tukaj tako tesnobno počutim. Do sedaj mi je bilo pri babici vedno prijetno.\" Glasno je zaklicala: \"Dobro jutro.\" Vendar odgovora ni dobila. Počasi je stopila do\ babičine postelje in odgrnila zavese. Zagledala je babico, ki je imela kapo poveznjeno skoraj čez cel\
obraz. Zelo čudna se ji je zdela.\
\"Joj, babica, kako velika ušesa imaš!\"\
\"Da te bolje slišim, otrok moj,\" je dobila v odgovor.\
\"Joj babica, kako velike oči imaš!\"\
\"Da te bolje vidim, draga moja.\"\
\"Toda babica, kako velike roke imaš!\"\
\"Da te lažje objamem.\"\
\"Oh. Babica. Kako grozno velika usta imaš!\"\
\"Da te lažje požrem!\"\
Še preden je volk do konca odgovoril, je že planil iz postelje\
in Rdečo kapico celo pogoltnil.\
Ko je volk pomiril svojo lakoto, je legel nazaj v posteljo in v trenutku zaspal. Ravno takrat je šel mimo\
 hiše lovec in si mislil: \"Kako glasno babica danes smrči. Grem in jo vprašam, če morda kaj potrebuje.\"\
Vstopil je v hišo in ko je pristopil k postelji, je zagledal, da tam namesto babice leži volk. \"Aha, tukaj" \
          " sem te našel!\" si je tiho rekel. \"Dolgo sem te iskal.\" Vanj je nameril svojo puško. Ravno, ko je hotel\
ustreliti, je pomislil, da je volk morda požrl babico in bi jo še lahko rešil. Zato je raje v roke vzel škarje\
in volku začel rezati trebuh. Že takoj po prvem rezu, je v volkovem trebuhu opazil nekaj rdečega. Hitro je naredil še dva reza in iz kosmatinčevega trebuha je ven skočila Rdeča kapica. Glasno je zajokala: \"Oh,\
kako me je bilo strah. V volkovem trebuhu je tako zelo temno.\" Takoj za tem je ven prilezla še babica,\
ki je komaj še dihala. Rdeča kapica je nato hitro nabrala nekaj velikih kamnov, s katerimi so napolnili\
volkov trebuh. Ko se je volk zbudil, je bil zelo žejen. Šel je k vodnjaku, da popije malo vode, vendar so\
bili kamni v njegovem trebuhu pretežki, zato je padel v vodnjak in poginil.\
Babica, Rdeča kapica in lovec so se tega zelo razveselili. Skupaj so pojedli potico, babica in lovec sta spila vino, ki ga je prinesla Rdeča kapica. Babica se je tako dobro okrepčala. Rdeča kapica pa si je mislila:\"Dokler bom živa, ne bom nikoli več skrenila s poti in tekala po gozdu, če mi bo mama tako rekla.\""

# Analyze using classla/stanza to get Document format
sl_text = corpus[6].text
data = sl_pipeline(sl_text)
print("extracted named entities".upper())
data = fix_ner(data)

print([" ".join([y.text for y in x.tokens]) for x in data.entities])


# Run named entity deduplication/resolution
deduplication_mapper, count = deduplicate_named_entities(data, count_entities=True)
print(count)
print("deduplication mapper".upper())
print(deduplication_mapper)
dedup_keys = deduplication_mapper.keys()


sentence_relations = get_relations_from_sentences(data, deduplication_mapper)
graph = create_graph_from_pairs(sentence_relations)
x = [print((x, y)) for y, x in sorted(zip(entity_importance_evaluation(graph), graph.nodes), reverse=True)]


# SVO triplet / event extraction
# Extract SVO triplets / events
events = e.eventify(sl_text)

print("SVO triplets / events".upper())
NER_containing_events = []
event_entity_count = {}
event_entities = []
event_tmp = []
for s, v, o in events:
    if isinstance(s, tuple) or isinstance(o, tuple):
        continue
    #print(s, "        ", v, " ", o)
    s_sim = find_similar(s, dedup_keys)
    o_sim = find_similar(o, dedup_keys)
    if s_sim is not None and o_sim is not None:
        print(s," ", v, " ", o)
        s = deduplication_mapper[s_sim]
        o = deduplication_mapper[o_sim]
        NER_containing_events.append((s, v, o))

    elif s != '<|EMPTY|>' and o != '<|EMPTY|>' and len(s.split()) < 3 and len(o.split()) < 3 and (s_sim is not None or o_sim is not None):
        event_entities.append(o)
        event_entities.append(s)
        try:
            event_entity_count[s] += 1
        except KeyError:
            event_entity_count[s] = 1
        try:
            event_entity_count[o] += 1
        except KeyError:
            event_entity_count[o] = 1
        #event_tmp.append((s, v, o))


deduplication_mapper, count = deduplicate_named_entities(event_entities, count_entities=True)
for s, v, o in event_tmp:
    NER_containing_events.append((deduplication_mapper[s], v, deduplication_mapper[o]))
print(deduplication_mapper)
print(count)
graph = create_graph_from_pairs(NER_containing_events)
print(graph.edges)

x = [print((x, y)) for y, x in sorted(zip(entity_importance_evaluation(graph), graph.nodes), reverse=True)]

#TODO: Replace with coreferences and map with deduplication mapper

EXTRACTED NAMED ENTITIES
['Gogi', 'Goge', 'Hana', 'Vajde', 'Klikot', 'Hanino sobo', 'Gapit', 'Teobald', 'Julio Gapit', 'Grbavec Teobald', 'Tarbula', 'Afra', 'Hanine fabule', 'Zolajevi noveli', 'Za noč ljubezni', 'Casanovi', 'Ijubimkanju', 'Tarbula', 'Afra', 'Klikot', 'TARBULA', 'AFRA', 'Juhčeva', 'TARBULA', 'AFRA', 'Vajdove hiše', 'Juta', 'Juta', 'Juta', 'TARBULA', 'JUTA', 'JUTA', 'Jontez', 'TARBULA', 'AFRA', 'TARBULA', 'Bog', 'Juta', 'Juta', 'TARBULA', 'Jute', 'Klef', 'Klef', 'Baron', 'Sort', 'Prisanka', 'Vajdi', 'Klef', 'Jelenom', 'Vajdov gost', 'TARBULA', 'Letoviščar', 'Bas', 'AFRA', 'Juta', 'TARBULA', 'Juta', 'Juta', 'Klefove navzočnosti', 'Vajdovim oglom', 'Klefa', 'Klef', 'Bog', 'Klef', 'AFRA', 'AFRA', 'AFRA', 'NADDAVKAR', 'Guštinova gospa', 'Guštinovo gospo', 'Tisovec', 'Jezus', 'Marija', 'Jezus', 'Marija', 'Jezus', 'Marija', 'Jezus', 'Marija', 'Jezus', 'Marija', 'Jezus', 'Marija', 'Jezus', 'Marija', 'TARBULA', 'AFRA', 'Tisovcu', 'AFRA', 'TARBULA', 'AFRA', 'cesti Gapit', 'Gizela

### English

In [5]:
# Initialize English stanza pipeline
stanza.download("en")
en_pipeline = stanza.Pipeline("en", processors='tokenize,ner, lemma, pos, depparse', use_gpu=True)

INFO:stanza:Downloading default packages for language: en (English)...
INFO:stanza:File exists: /home/erikkastelec/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /home/erikkastelec/stanza_resources.


INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| ner       | ontonotes |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [15]:
# Analyze using classla/stanza to get Document format
data = en_pipeline(en_text)
print("extracted named entities".upper())
print([" ".join([y.text for y in x.tokens]) for x in data.entities])

# Run named entity deduplication/resolution
deduplication_mapper = deduplicate_named_entities(data)
print("deduplication mapper".upper())
print(deduplication_mapper)

e = Eventify(language="en")
# Extract SVO triplets / events
events = e.eventify(en_text)
print("SVO triplets / events".upper())
print(events)
#TODO: Replace with coreferences and map with deduplication mapper

EXTRACTED NAMED ENTITIES
['Bingley', 'four or five thousand', 'Bennet', 'one', 'one', 'Bingley', 'five', 'Bingley', 'one', 'William', 'Lady Lucas', 'Bingley', 'Lizzy', 'Lizzy', 'Jane', 'Lydia', 'Lizzy', 'Bennet']
DEDUPLICATION MAPPER
{'Bingley': 'Bingley', 'four or five thousand': 'four or five thousand', 'Bennet': 'Bennet', 'one': 'one', 'five': 'five', 'William': 'William', 'Lady Lucas': 'Lady Lucas', 'Lizzy': 'Lizzy', 'Jane': 'Jane', 'Lydia': 'Lydia'}
SVO TRIPLETS / EVENTS
[('he', 'single', '<|EMPTY|>'), ('it', 'affect', 'them'), ('his wife', 'dear Mr. Bennet', '<|EMPTY|>'), ('his wife', 'reply', 'My dear Mr. Bennet'), ('his wife', 'reply', 'how you be so tiresome'), ('You', 'know', ('I', 'think', '<|EMPTY|>')), ('you', 'nonsense talk', '<|EMPTY|>'), ('he', 'fall', '<|EMPTY|>'), ('you', 'visit', 'him'), ('you', '', '<|EMPTY|>'), ('he', 'come', '<|EMPTY|>'), ('I', 'see', 'occasion for that'), ('You', 'go', '<|EMPTY|>'), ('girls', 'go', '<|EMPTY|>'), ('you', 'send', 'them'), ('Mr. Bin